### 训练中文点选验证码-类型1

使用yolo11n模型进行目标检测

使用vgg16模型训练孪生神经网络

In [1]:
import sys
import os

# 把当前工作目录添加到sys.path中
sys.path.append(os.path.abspath(".."))

In [2]:
# 数据集
import my_datasets
from datasets.arrow_dataset import Dataset


dataset: Dataset = my_datasets.get_dataset("captcha_chinese_click_1")
print(dataset)

/home/qsr/.conda/envs/py313/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-02 02:48:34,399 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from captcha_chinese_click_1. Please make sure that you can trust the external codes.
2025-04-02 02:48:35,133 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from Amorter/captcha_chinese_click_1. Please make sure that you can trust the external codes.
2025-04-02 02:48:35,134 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from Amorter/captcha_chinese_click_1. Please make sure that you can trust the external codes.
2025-04-02 02:48:35,134 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from Amorter/captcha_chinese_click_1. Please make sure that you can trust the extern

ConnectionError: Couldn't reach https://www.modelscope.cn/api/v1/datasets/Amorter/captcha_chinese_click_1/repo?Source=SDK&Revision=master&FilePath=dataset%2Ftrain%2F1021.jpg (ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='www.modelscope.cn', port=443): Read timed out. (read timeout=100)")))

#### 划分训练集和测试集

In [ ]:
# 数据总共451张验证码图片，比较少，所以测试集比例选择了10%

train_test_dataset = dataset.train_test_split(test_size=0.1, shuffle=True)
train_dataset = train_test_dataset['train']
test_dataset = train_test_dataset['test']
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['image', 'objects'],
    num_rows: 405
})
Dataset({
    features: ['image', 'objects'],
    num_rows: 46
})


#### 数据集转换为yolo格式

In [ ]:
from PIL import Image
import os


def toYolo(dataset: Dataset, output_dir):
    # 设置输出目录的位置
    images_dir = os.path.join(output_dir, "images")  # 图片保存目录
    labels_dir = os.path.join(output_dir, "labels")  # 标签保存目录

    # 确保输出目录及其子目录存在
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for idx, data in enumerate(dataset):
        image: Image = data['image']
        objects: object = data['objects']
        bbox: list[list[int]] = objects['bbox']
        categories: list[int] = objects['categories']

        # 获取图像的宽度和高度
        img_width, img_height = image.size

        # 获取图片文件名（或生成默认名）
        image_filename = idx + '.jpg'
        label_filename = idx + '.txt'

        # 构造图片和标签的保存路径
        image_filepath = os.path.join(images_dir, image_filename)
        label_filepath = os.path.join(labels_dir, label_filename)

        # 保存图片到 images 文件夹
        image.save(image_filepath)

        # 保存标签到 labels 文件夹
        with open(label_filepath, 'w') as f:  # 打开标签文件准备写入
            for idx, box in enumerate(bbox):
                x_min, y_min, x_max, y_max = box
                category = categories[idx]

                # 将边界框转换为YOLO格式：(class_id, x_center, y_center, width, height)
                x_center = (x_min + x_max) / 2 / img_width
                y_center = (y_min + y_max) / 2 / img_height
                width = (x_max - x_min) / img_width
                height = (y_max - y_min) / img_height

                # 写入标签文件，格式为：class_id x_center y_center width height
                f.write(f"{category} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=344x384 at 0x7DEE0C145F30>, 'objects': {'bbox': [[227, 4, 290, 68], [103, 78, 168, 146], [65, 221, 129, 288], [82, 349, 107, 380], [55, 347, 83, 377], [206, 213, 269, 281], [0, 348, 26, 377], [26, 348, 55, 375]], 'categories': [0, 3, 1, 0, 1, 2, 3, 2]}}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=344x384 at 0x7DEE0C145940>, 'objects': {'bbox': [[106, 23, 163, 81], [81, 164, 143, 223], [29, 348, 56, 377], [0, 347, 30, 376]], 'categories': [0, 1, 0, 1]}}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=344x384 at 0x7DEE0C145F30>, 'objects': {'bbox': [[7, 225, 72, 287], [150, 246, 214, 309], [214, 105, 274, 165], [111, 120, 177, 189], [0, 348, 29, 377], [26, 348, 56, 378], [80, 348, 108, 379], [56, 348, 82, 380]], 'categories': [1, 0, 2, 3, 3, 2, 0, 1]}}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=344x384 at 0x7DEE0C145940>, 'objects': {'bbox': [[28, 347, 54, 375], [1

UnidentifiedImageError: cannot identify image file '/home/qsr/.cache/modelscope/hub/datasets/downloads/7de1e9eb59fb0f66032beee5d5ce24e689de72bbdbf81244a8cffbd42c3f4602'